In [11]:
import pyoti.core as coti
import pyoti.sparse as oti
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern Roman",
    "font.size": 12,
    "font.sans-serif": ["Helvetica"]
})

In [12]:
def fun2(X,r,m,b):
    #INTERSECTION BETWEEN CURVES.
    f = oti.zeros((2,1))
    x = X[0,0].copy()
    y = X[1,0].copy()
    f[0,0] = x**2 + y**2 - r**2 
    f[1,0] = y - ( m * x + b )
    return f

def Jfun2(X,func,verbose,r,m,b):
    #INTERSECTION BETWEEN CURVES.
    J = oti.zeros((2,2))
    x = X[0,0].copy()
    y = X[1,0].copy()
    J[0,0] = 2 * x 
    J[1,0] = - m 
    J[0,1] = 2 * y 
    J[1,1] = 1
    return J

def fun2_curves(x,r,m,b):
    #CURVES.
    y = oti.zeros((x.shape[0],3))
    y[:,0] =  oti.sqrt(r**2 - x**2)
    y[:,1] = -oti.sqrt(r**2 - x**2)
    y[:,2] = m * x + b
    return y


def intersect_pts(r,m,b,alg=oti):
    #CURVES.
        
    #     x[0,0] = (-m*b + oti.sqrt( (m*b)**2 - (m+1)*(b**2-r**2) ) )/( (m+1) )
    #     x[1,0] = (-m*b - oti.sqrt( (m*b)**2 - (m+1)*(b**2-r**2) ) )/( (m+1) )

    x1 = (-b*m + alg.sqrt(-b**2 + m**2*r**2 + r**2))/(m**2 + 1)
    y1 = m * x1 + b
    x2 = (-b*m - alg.sqrt(-b**2 + m**2*r**2 + r**2))/(m**2 + 1)
    y2 = m * x2 + b
        
    return x1,y1,x2,y2

In [13]:
# utils
def newton_otisis(func, jaco, x, tol, maxiter=50, args=(), verbose=False,eps = 1e-16):
    error = 1e30
    n = 0
    nx = x.shape[0]
    dx = oti.zeros( (nx,1), order=x.order)
    
    argv = (func,verbose)+args
    while error > tol:

        if n==maxiter:
            print("Did not converge. ", maxiter, "iters.")
            break
        if verbose:
            print("Iteration {0}".format(n))
        mJ = -jaco(x,*argv)
#         print(mJ.shape)
        f_eval = func(x,*args)
#         print(f_eval.shape)

        
        dx = oti.dot(oti.inv(mJ),f_eval)
        x += dx
        error = oti.norm(dx)/(oti.norm(x)+eps)
        error = (error).get_deriv(0)
        
        if verbose:
            print("\nf")
            print(f_eval)
            print("\nJ")
            print(-mJ)
            print('\ndx')
            print(dx)
            print("\nx i+1")
            print(x)
            print("Error")
            print("{0:.10f}".format(error))
            print("\n"+100*"=")
        n += 1
    print("niter: ",n)
#     print("rel_err:",(oti.norm(dx)/oti.norm(x)).get_deriv(0))
    return x

def j_DF (x, func, *args):
    h = 1e-8
    nx = x.shape[0]
    J = oti.zeros((nx,nx), order=x.order)
    for i in range(nx):
        xh = x.copy()
        xh[i,0]+= h
        dfdxi = (func(xh,*args) - func(x,*args))/h
        for j in range(nx):
            J[j,i] = dfdxi[j,0]
    return J

def j_oti (x, func,verbose, *args):
    nx = x.shape[0]
    active_bases = func(x,*args).get_active_bases()
    if len(active_bases)==0:
        next_base = 1 
    else:
        next_base = max(active_bases) + 1
    # end if 
    
    e_x = oti.zeros((nx,1), order=(x.order+1))
    for i in range(nx):
        e_x[i] = oti.e(next_base + i, order = (x.order+1))  
    # end for 
    J = oti.zeros((nx,nx), order=x.order)
    fun = func(x+e_x, *args) 
    if verbose:
        print('\nf for jacobian')
        print(fun)
    for n in range(nx):
        J_col = fun.extract_deriv(next_base + n)
        
        for j in range(nx):
            J_col = J_col.truncate(next_base + j)
        # end for 
        for m in range(nx):
            J[m,n] = J_col[m,0]
        # end for 
    # end for 
    return J

In [25]:
oti.set_printoptions(float_format='g')

In [95]:
order = 3
x0 = oti.array([1,1])
r  = 4+oti.e(1, order = order)
m  = 3+oti.e(2, order = order)
b = -4+oti.e(3, order = order)

In [96]:
# Solve the real system.
X = newton_otisis(fun2,j_oti,x0,1e-6, args = (r,m,b), verbose=False)
X

niter:  6


matso< shape: (2, 1), 
 - Column 0
(0,0) 2.4 + 0.333333 * e([1]) - 0.64 * e([2]) - 0.266667 * e([3]) - 0.00462963 * e([[1,2]]) - 0.111111 * e([1,2]) + 0.144 * e([[2,2]]) - 0.00925926 * e([1,3]) + 0.0488889 * e([2,3]) - 0.00462963 * e([[3,2]]) + 0.00128601 * e([[1,3]]) + 0.00462963 * e([[1,2],2]) + 0.037037 * e([1,[2,2]]) - 0.0224 * e([[2,3]]) + 0.00270062 * e([[1,2],3]) + 0.00925926 * e([1,2,3]) + 0.00103704 * e([[2,2],3]) + 0.00154321 * e([1,[3,2]]) + 0.00462963 * e([2,[3,2]]) + 0.000128601 * e([[3,3]])
(1,0) 3.2 + 1 * e([1]) + 0.48 * e([2]) + 0.2 * e([3]) - 0.0138889 * e([[1,2]]) + 5.04035e-17 * e([1,2]) - 0.208 * e([[2,2]]) - 0.0277778 * e([1,3]) - 0.12 * e([2,3]) - 0.0138889 * e([[3,2]]) + 0.00385802 * e([[1,3]]) + 0.00925926 * e([[1,2],2]) + 2.97376e-17 * e([1,[2,2]]) + 0.0768 * e([[2,3]]) + 0.00810185 * e([[1,2],3]) + 0.0185185 * e([1,2,3]) + 0.052 * e([[2,2],3]) + 0.00462963 * e([1,[3,2]]) + 0.00925926 * e([2,[3,2]]) + 0.000385802 * e([[3,3]])
>

In [97]:
Xr = newton_otisis(fun2,j_oti,x0,1e-6, args = (r.real,m.real,b.real), verbose=False)
Xr

niter:  6


matso< shape: (2, 1), 
 - Column 0
(0,0) 2.4
(1,0) 3.2
>

## First order derivatives:

In [98]:
Xev = Xr.copy()
Xev[0,0] = Xev[0,0] + oti.e(4,order=order)
Xev[1,0] = Xev[1,0] + oti.e(5,order=order)
residual = fun2(Xev,r,m,b)

r1 = residual[0,0].copy()
r2 = residual[1,0].copy()
print(r1, r2)

# Xsolve
J = j_oti(Xr,fun2,False,r.real,m.real,b.real)
Jinv = oti.inv(J)
Jinv

-1.77636e-15 - 8 * e([1]) + 4.8 * e([4]) + 6.4 * e([5]) - 1 * e([[1,2]]) + 1 * e([[4,2]]) + 1 * e([[5,2]]) 4.44089e-16 - 2.4 * e([2]) - 1 * e([3]) - 3 * e([4]) + 1 * e([5]) - 1 * e([2,4])


matso< shape: (2, 2), 
 - Column 0
(0,0) 0.0416667
(1,0) 0.125
 - Column 1
(0,1) -0.266667
(1,1) 0.2
>

In [99]:
R1.truncate(4).truncate(5)

matso< shape: (2, 1), 
 - Column 0
(0,0) 0 - 8 * e([1])
(1,0) 0 - 2.4 * e([2]) - 1 * e([3])
>

In [100]:
Xsol = oti.zeros((2,1))

In [101]:
sol_order = 1
R1 = oti.zeros((2,1))
R1[0,0] = r1.rom_eval_object([1,2,3,4,5],[oti.e(1,order=order),oti.e(2,order=order),oti.e(3,order=order),0*oti.e(4,order=order),0*oti.e(5,order=order)])
R1[1,0] = r2.rom_eval_object([1,2,3,4,5],[oti.e(1,order=order),oti.e(2,order=order),oti.e(3,order=order),0*oti.e(4,order=order),0*oti.e(5,order=order)])
R1 = R1.truncate_order(1)
R1 = R1.truncate(4).truncate(5)
R1 = R1.get_order_im(sol_order)
X1 = oti.dot(Jinv,-R1)
Xsol += X1
X1

matso< shape: (2, 1), 
 - Column 0
(0,0) 0 + 0.333333 * e([1]) - 0.64 * e([2]) - 0.266667 * e([3])
(1,0) 0 + 1 * e([1]) + 0.48 * e([2]) + 0.2 * e([3])
>

In [102]:
sol_order = 2
R2 = oti.zeros((2,1))
R2[0,0] = r1.rom_eval_object([1,2,3,4,5],[oti.e(1,order=order),oti.e(2,order=order),oti.e(3,order=order),X1[0,0],X1[1,0]])
R2[1,0] = r2.rom_eval_object([1,2,3,4,5],[oti.e(1,order=order),oti.e(2,order=order),oti.e(3,order=order),X1[0,0],X1[1,0]])
R2 = R2.truncate_order(sol_order)
R2 = R2.truncate(4).truncate(5)
R2 = R2.get_order_im(sol_order)
X2 = oti.dot(Jinv,-R2)
Xsol += X2
X2

matso< shape: (2, 1), 
 - Column 0
(0,0) 0 - 0.00462963 * e([[1,2]]) - 0.111111 * e([1,2]) + 0.144 * e([[2,2]]) - 0.00925926 * e([1,3]) + 0.0488889 * e([2,3]) - 0.00462963 * e([[3,2]])
(1,0) 0 - 0.0138889 * e([[1,2]]) - 1.38778e-17 * e([1,2]) - 0.208 * e([[2,2]]) - 0.0277778 * e([1,3]) - 0.12 * e([2,3]) - 0.0138889 * e([[3,2]])
>

In [103]:
sol_order = 3
R3 = oti.zeros((2,1))
R3[0,0] = r1.rom_eval_object([1,2,3,4,5],[oti.e(1,order=order),oti.e(2,order=order),oti.e(3,order=order),Xsol[0,0],Xsol[1,0]])
R3[1,0] = r2.rom_eval_object([1,2,3,4,5],[oti.e(1,order=order),oti.e(2,order=order),oti.e(3,order=order),Xsol[0,0],Xsol[1,0]])
R3 = R3.truncate_order(sol_order)
R3 = R3.truncate(4).truncate(5)
R3 = R3.get_order_im(sol_order)
X3 = oti.dot(Jinv,-R3)
Xsol += X3
X3

matso< shape: (2, 1), 
 - Column 0
(0,0) 0 + 0.00128601 * e([[1,3]]) + 0.00462963 * e([[1,2],2]) + 0.037037 * e([1,[2,2]]) - 0.0224 * e([[2,3]]) + 0.00270062 * e([[1,2],3]) + 0.00925926 * e([1,2,3]) + 0.00103704 * e([[2,2],3]) + 0.00154321 * e([1,[3,2]]) + 0.00462963 * e([2,[3,2]]) + 0.000128601 * e([[3,3]])
(1,0) 0 + 0.00385802 * e([[1,3]]) + 0.00925926 * e([[1,2],2]) + 3.46945e-18 * e([1,[2,2]]) + 0.0768 * e([[2,3]]) + 0.00810185 * e([[1,2],3]) + 0.0185185 * e([1,2,3]) + 0.052 * e([[2,2],3]) + 0.00462963 * e([1,[3,2]]) + 0.00925926 * e([2,[3,2]]) + 0.000385802 * e([[3,3]])
>

In [104]:
X.get_order_im(3)

matso< shape: (2, 1), 
 - Column 0
(0,0) 0 + 0.00128601 * e([[1,3]]) + 0.00462963 * e([[1,2],2]) + 0.037037 * e([1,[2,2]]) - 0.0224 * e([[2,3]]) + 0.00270062 * e([[1,2],3]) + 0.00925926 * e([1,2,3]) + 0.00103704 * e([[2,2],3]) + 0.00154321 * e([1,[3,2]]) + 0.00462963 * e([2,[3,2]]) + 0.000128601 * e([[3,3]])
(1,0) 0 + 0.00385802 * e([[1,3]]) + 0.00925926 * e([[1,2],2]) + 2.97376e-17 * e([1,[2,2]]) + 0.0768 * e([[2,3]]) + 0.00810185 * e([[1,2],3]) + 0.0185185 * e([1,2,3]) + 0.052 * e([[2,2],3]) + 0.00462963 * e([1,[3,2]]) + 0.00925926 * e([2,[3,2]]) + 0.000385802 * e([[3,3]])
>

In [69]:
# Compute Jacobian at this iteration:

J = j_oti(Xr,fun2,False,r.real,m.real,b.real)

residual = fun2(Xr,r,m,b)
print(residual)
dresdr = oti.array(residual.get_deriv(1))
dxdr = -oti.dot(Jinv,oti.array(residual.get_deriv(1)))
dxdm = -oti.dot(Jinv,oti.array(residual.get_deriv(2)))
dxdb = -oti.dot(Jinv,oti.array(residual.get_deriv(3)))


matso< shape: (2, 1), 
 - Column 0
(0,0) -0.0000 - 8.0000 * e([1]) - 1.0000 * e([[1,2]])
(1,0) 0.0000 - 2.4000 * e([2]) - 1.0000 * e([3])
>


In [62]:
print(dxdr.real)
print(X.get_deriv(1).real)
oti.norm(oti.array(X.get_deriv(1))-dxdr).real

[[0.33333333]
 [1.        ]]
[[0.33333333]
 [1.        ]]


1.1102230246251565e-16

In [64]:
print(dxdm.real)
print(X.get_deriv(2))
oti.norm(oti.array(X.get_deriv(2))-dxdm).real

[[-0.64]
 [ 0.48]]
[[-0.64]
 [ 0.48]]


2.2887833992611187e-16

In [67]:
print(dxdb.real)
print(X.get_deriv(3))
oti.norm(oti.array(X.get_deriv(3))-dxdb).real

[[-0.26666667]
 [ 0.2       ]]
[[-0.26666667]
 [ 0.2       ]]


5.551115123125783e-17

## Second order derivatives

In [ ]:
rhs = oti.array(residual.get_deriv([[1,2]])) + 
d2xdrr = -oti.dot(Jinv,-)

In [9]:
X.get_order_im(2)

matso< shape: (2, 1), 
 - Column 0
(0,0) 0.0000 - 0.0046 * e([[1,2]]) - 0.1111 * e([1,2]) + 0.1440 * e([[2,2]]) - 0.0093 * e([1,3]) + 0.0489 * e([2,3]) - 0.0046 * e([[3,2]])
(1,0) 0.0000 - 0.0139 * e([[1,2]]) + 0.0000 * e([1,2]) - 0.2080 * e([[2,2]]) - 0.0278 * e([1,3]) - 0.1200 * e([2,3]) - 0.0139 * e([[3,2]])
>

In [10]:
residual

NameError: name 'residual' is not defined